In [ ]:
!pip install nltk==3.3

In [ ]:
import nltk
nltk.download('twitter_samples')

from nltk.corpus import twitter_samples


[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


In [ ]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(tweet_tokens[0])


['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']


In [ ]:
#Normalizando (stemming e lematização)

nltk.download('wordnet') #banco de dados lexical para o idioma inglês que ajuda o script a determinar a palavra base
nltk.download('averaged_perceptron_tagger') #recurso para determinar o contexto de uma palavra em uma frase

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
from nltk.tag import pos_tag

tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(pos_tag(tweet_tokens[0]))

# NNP: Substantivo, próprio, singular
# NNSubstantivo, comum, singular ou de massa
# IN: Preposição ou conjunção, subordinação
# VBG: Partícula verbal, gerúndria ou presente
# VBN: Verbo, particípio passado


[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer 

def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence 

#print(lemmatize_sentence(tweet_tokens[0]))

In [ ]:
import re, string 
def remove_noise(tweet_tokens, stop_words = ()):
    cleaned_tokens = []
    for token, tag in pos_tag(tweet_tokens):
        token = re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\'(?:%[0-9a-fA-F][0-9a-fA-F]))+",'', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
          cleaned_tokens.append(token.lower())
    return cleaned_tokens


In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

stop_words = stopwords.words('english') 
print(remove_noise(tweet_tokens[0], stop_words))


['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']


In [ ]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')
positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []
for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

print(positive_tweet_tokens[500])
print(positive_cleaned_tokens_list[500])


['Dang', 'that', 'is', 'some', 'rad', '@AbzuGame', '#fanart', '!', ':D', 'https://t.co/bI8k8tb9ht']
['dang', 'rad', '#fanart', ':d']


In [ ]:
#Determinando densidade

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token 
all_pos_words = get_all_words(positive_cleaned_tokens_list)


In [ ]:
from nltk import FreqDist 
freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))


[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [ ]:
# Preparando dados para o modelo

# Convertendo tokens num dicionário
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens) 
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)


In [ ]:
# embaralhando os dados com shuffle pra treinar 70-30
import random 
positive_dataset = [(tweet_dict, "Positive")for tweet_dict in positive_tokens_for_model] 
negative_dataset = [(tweet_dict, "Negative")for tweet_dict in negative_tokens_for_model] 
dataset = positive_dataset + negative_dataset 
random.shuffle(dataset) 
train_data = dataset[:7000]
test_data = dataset[7000:]

In [ ]:
# criando e testando o modelo
from nltk import classify
from nltk import NaiveBayesClassifier

classifier = NaiveBayesClassifier.train(train_data) 
print("Accuracy is:", classify.accuracy(classifier, test_data)) 
print(classifier.show_most_informative_features(10))


Accuracy is: 0.995
Most Informative Features
                      :) = True           Positi : Negati =   1653.9 : 1.0
                     sad = True           Negati : Positi =     24.8 : 1.0
                     bam = True           Positi : Negati =     23.2 : 1.0
                     x15 = True           Negati : Positi =     18.2 : 1.0
                followed = True           Negati : Positi =     15.3 : 1.0
                  arrive = True           Positi : Negati =     13.7 : 1.0
                    blog = True           Positi : Negati =     12.4 : 1.0
                    miss = True           Negati : Positi =     12.1 : 1.0
                 perfect = True           Positi : Negati =     11.8 : 1.0
                   great = True           Positi : Negati =     11.5 : 1.0
None


In [ ]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.tokenize import word_tokenize 

custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."
custom_tweet_2 = "Thank you for sending my baggage to CityX and flying me to CityY at the same time. Brilliant service. #thanksGenericAirline"

custom_tokens = remove_noise(word_tokenize(custom_tweet)) 
custom_tokens_2 = remove_noise(word_tokenize(custom_tweet_2))

print(classifier.classify(dict([token, True] for token in custom_tokens)))
print(classifier.classify(dict([token, True] for token in custom_tokens_2)))

Negative
Positive


In [ ]:
# Perguntinha final
custom_tweety = "I ordered just once from TerribleCo, they screwed up gain." 
custom_tokensy = remove_noise(word_tokenize(custom_tweety)) 
print(classifier.classify(dict([token, True] for token in custom_tokensy)))


Positive
